In [1]:
import numpy as np 
import pandas as pd
import sklearn
# !conda install requests
import requests
# !conda install beautifulsoup4
from bs4 import BeautifulSoup 
print("done!")

done!


In [2]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(url)
print("done!")

done!


In [3]:
table = BeautifulSoup(r.text, 'html.parser')

In [4]:
print(r.headers['Content-Type'])

text/html; charset=UTF-8


In [5]:
rows = table.find_all('tr')

In [6]:
columns = [v.text.replace('\n', '') for v in rows[0].find_all('th')]
print(columns)

['Postal Code', 'Borough', 'Neighbourhood']


In [7]:
df = pd.DataFrame(columns=columns)
df

,Postal Code,Borough,Neighbourhood


In [8]:
for i in range(len(rows)):
    # made list with all td 
    tds = rows[i].find_all('td')
#     print(tds,i)
    # deleted blank in tds 
    if len(tds) == len(columns):
        # in every row, all data are stored in values
        values = [ td.text.replace('\n', '').replace('\xa0', ' ') for td in tds ]
        # turned values into pd.series v
        df = df.append(pd.Series(values, index=columns), ignore_index= True)
df.head(183)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [9]:
# pandas.read_html() is also useful


### The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [10]:
df


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [11]:
print(df.columns)


Index(['Postal Code', 'Borough', 'Neighbourhood'], dtype='object')


### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [12]:
df2=df[df['Borough']!="Not assigned"]# chose rows with valid name in Borough column
df2=df2.reset_index(drop=True)
df2

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


### More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [13]:
type(df2.duplicated(subset="Postal Code"))

pandas.core.series.Series

In [14]:
for i in range(len(df2.duplicated(subset="Postal Code"))):
    if  df2.duplicated(subset="Postal Code")[i]==True:
        print (df2.duplicated(subset="Postal Code")[i])


### there is no duplicate in Postal Code

### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [15]:
len(df)

180

In [16]:
for i in range(len(df2)):
    if df2["Neighbourhood"][i]=="Not assigned":
        df2["Neighbourhood"][i]==df2["Borough"][i]
df2

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


### In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [17]:
df2.shape

(103, 3)